In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
import pandas as pd

In [3]:
# list paths in 'data/citylearn_challenge_2022_phase_1/' that start with Build
# and end with .csv
files = [f for f in os.listdir('data/citylearn_challenge_2022_phase_1/') if f.startswith('Build') and f.endswith('.csv')]
files

['Building_4.csv',
 'Building_5.csv',
 'Building_1.csv',
 'Building_2.csv',
 'Building_3.csv']

In [4]:
weather = pd.read_csv('data/citylearn_challenge_2022_phase_1/weather.csv')
weather

,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],6h Prediction Relative Humidity [%],12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2]
0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,81.0,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0
1,20.1,79.0,0.0,0.0,19.4,22.8,19.4,79.0,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0
2,19.7,78.0,0.0,0.0,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0
3,19.3,78.0,0.0,0.0,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0
4,18.9,78.0,0.0,0.0,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,20.6,84.0,26.0,130.0,20.1,19.4,20.6,79.0,79.0,73.0,0.0,201.0,27.0,0.0,444.0,143.0
8756,21.1,81.0,0.0,0.0,19.7,21.1,20.0,78.0,73.0,76.0,0.0,420.0,0.0,0.0,592.0,0.0
8757,21.7,79.0,0.0,0.0,19.3,22.2,20.6,78.0,71.0,70.0,0.0,554.0,0.0,0.0,491.0,0.0
8758,21.3,76.0,0.0,0.0,18.9,21.7,20.6,78.0,73.0,73.0,0.0,778.0,0.0,0.0,734.0,0.0


In [20]:
scens_point = pd.read_csv('debug_logs/scenarios_point_recurrent_9000_1.csv')
# rename columns from +1 to +24 to +0 to +23
scens_point.columns = [f'{int(c.split("+")[1].split("h")[0])-1}' if c.startswith('+') else c for c in scens_point.columns]
# add plus sign to column names
scens_point.columns = [f'forecast+{c}' if c.isdigit() else c for c in scens_point.columns]
# drop scenario column
scens_point = scens_point.drop(columns=['scenario'])
# save to csv
scens_point.to_csv('point_forecast.csv', index=True)
scens_point

,time_step,building,forecast+0,forecast+1,forecast+2,forecast+3,forecast+4,forecast+5,forecast+6,forecast+7,...,forecast+14,forecast+15,forecast+16,forecast+17,forecast+18,forecast+19,forecast+20,forecast+21,forecast+22,forecast+23
0,0,0,0.320999,-0.100628,-0.146914,-0.033779,0.038974,0.026710,-0.667358,-1.638209,...,-2.368901,-2.023600,-1.635879,-1.050545,-0.229833,0.072593,0.059910,-0.026226,-0.026191,-0.071121
1,0,1,0.306807,0.042247,-0.178280,-0.063702,-0.103141,-0.112332,-0.632428,-1.359930,...,-1.907470,-1.648721,-1.358184,-0.919567,-0.304571,-0.077950,-0.087453,-0.151999,-0.151972,-0.185641
2,0,2,-0.597894,-0.842980,-0.874195,-0.797897,-0.748832,-0.757103,-1.225185,-1.879930,...,-2.372712,-2.139839,-1.878359,-1.483608,-0.930116,-0.726159,-0.734713,-0.792803,-0.792779,-0.823081
3,0,3,0.535133,-0.168215,-0.442046,-0.613359,-0.601909,-0.610345,-1.087744,-1.755523,...,-2.258113,-2.020605,-1.753920,-1.351311,-0.786802,-0.578785,-0.587509,-0.646756,-0.646731,-0.677636
4,0,4,-0.320111,-0.631659,-0.663565,-0.585579,-0.535429,-0.543883,-1.022317,-1.691543,...,-2.195222,-1.957200,-1.689937,-1.286455,-0.720723,-0.512255,-0.520997,-0.580373,-0.580348,-0.611320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,8759,0,1.937666,1.160807,0.809839,0.773878,0.765133,0.805523,0.704703,0.306982,...,0.093729,0.552745,0.965864,1.535550,1.926750,1.757545,1.887569,2.158329,2.417111,2.347384
43796,8759,1,1.862208,0.859199,0.823265,0.820058,0.886900,0.611825,0.450529,0.210049,...,0.319728,0.585658,0.667099,0.773370,0.933223,1.185214,1.226577,1.482469,1.410149,1.685158
43797,8759,2,0.023599,0.014475,0.040063,0.062488,0.125830,0.149612,0.149770,0.066329,...,-0.727149,-0.263699,0.041003,0.269161,0.544257,0.750464,0.889015,0.972730,1.022416,0.496470
43798,8759,3,1.932673,1.427357,1.265587,1.005865,0.735524,0.833445,0.616311,0.465955,...,0.101265,0.137206,0.393378,0.805132,1.100058,1.154574,0.969148,0.881046,1.110139,1.293626


In [37]:
pv_capacity = [4.0, 4.0, 4.0, 5.0, 4.0]
# loop through csv files and extract consumption and generation data
all_buildings = pd.DataFrame()
for file in files:
    df = pd.read_csv('data/citylearn_challenge_2022_phase_1/' + file)
    filename = file.split('.')[0].split('_')[1]
    df = pd.concat([df, weather], axis=1)
    df['time_step'] = df.index
    df['building'] = int(filename) - 1
    df['PV capacity [kW]'] = pv_capacity[int(filename[-1])-1]
    df.drop(['Daylight Savings Status','DHW Heating [kWh]',
       'Cooling Load [kWh]', 'Heating Load [kWh]'], axis=1, inplace=True)
    # Remove column names containing the word 'Prediction'
    df = df[[col for col in df.columns if 'Prediction' not in col]]
    all_buildings = pd.concat([all_buildings, df], axis=0)
all_buildings.rename(columns={'Hour': 'hour', 'Month': 'month', 'Day Type': 'day_type'}, inplace=True)
#move time_step and building to front
cols = all_buildings.columns.tolist()
cols = cols[-2:] + cols[:-2]
all_buildings = all_buildings[cols]
# remove columns with no data
all_buildings = all_buildings.dropna(axis=1, how='all')
all_buildings


,building,PV capacity [kW],month,hour,day_type,Equipment Electric Power [kWh],Solar Generation [W/kW],Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],time_step
0,3,5.0,7.0,24.0,7.0,2.819150,0.000000,20.0,84.0,0.0,0.0,0
1,3,5.0,8.0,1.0,1.0,1.928167,0.000000,20.1,79.0,0.0,0.0,1
2,3,5.0,8.0,2.0,1.0,0.475817,0.000000,19.7,78.0,0.0,0.0,2
3,3,5.0,8.0,3.0,1.0,0.458233,0.000000,19.3,78.0,0.0,0.0,3
4,3,5.0,8.0,4.0,1.0,0.446017,0.000000,18.9,78.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2,4.0,7.0,19.0,1.0,1.472417,13.670833,20.6,84.0,26.0,130.0,8755
8756,2,4.0,7.0,20.0,1.0,1.075767,0.000000,21.1,81.0,0.0,0.0,8756
8757,2,4.0,7.0,21.0,1.0,1.162883,0.000000,21.7,79.0,0.0,0.0,8757
8758,2,4.0,7.0,22.0,1.0,1.107300,0.000000,21.3,76.0,0.0,0.0,8758


In [39]:
# save to csv
all_buildings.to_csv('all_buildings_phase1.csv', index=True)